# Import libraries

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import astropy.convolution as krn
import scipy.stats as stats
import sys

In [36]:
def makeHeat(screenRes, xPos, yPos):
        xMax = screenRes[0]
        yMax = screenRes[1]
        xMin = 0
        yMin = 0
        kernelPar = 50

        # Input handeling
        xlim = np.logical_and(xPos < xMax, xPos > xMin)
        ylim = np.logical_and(yPos < yMax, yPos > yMin)
        xyLim = np.logical_and(xlim, ylim)
        dataX = xPos[xyLim]
        dataX = np.floor(dataX)
        dataY = yPos[xyLim]
        dataY = np.floor(dataY)

        # initiate map and gauskernel
        gazeMap = np.zeros([int((xMax-xMin)),int((yMax-yMin))])+0.0001
        gausKernel = krn.Gaussian2DKernel(kernelPar)

        # Rescale the position vectors (if xmin or ymin != 0)
        dataX -= xMin
        dataY -= yMin

        # Now extract all the unique positions and number of samples
        xy = np.vstack((dataX, dataY)).T
        uniqueXY, idx, counts = uniqueRows(xy)
        uniqueXY = uniqueXY.astype(int)
        # populate the gazeMap
        gazeMap[uniqueXY[:,0], uniqueXY[:,1]] = counts

        # Convolve the gaze with the gauskernel
        heatMap = np.transpose(krn.convolve_fft(gazeMap,gausKernel))
        heatMap = heatMap/np.max(heatMap)

        return heatMap

def uniqueRows(x):
    y = np.ascontiguousarray(x).view(np.dtype((np.void, x.dtype.itemsize * x.shape[1])))
    _, idx, counts = np.unique(y, return_index=True, return_counts = True)
    uniques = x[idx]
    return uniques, idx, counts


def np_euclidean_distance(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sqrt(np.sum(np.square(y_pred - y_true), axis=-1))



In [37]:
sys.path.append('./FixationDetection')
from I2MC import runI2MC

# Path to data folders
# path_to_folders = 'C:/Users/artem/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_MullerLyer'
path_to_folders = 'D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_MullerLyer'

# get all folder names
folder_names = os.listdir(path_to_folders)

# read and process original datafile for each participant
for fn in folder_names:
    path = os.path.join(path_to_folders, fn, fn+'_record.csv')       
        
    df = pd.read_csv(path)      
    


In [38]:
# order frames and drop duplicate samples (with same sampleTime)
df = df[df.fName.notna()]
df = df[df.frameNr.notna()]
df = df.apply(pd.to_numeric, errors='ignore')

df = df.sort_values('frameNr')
df = df.reset_index(drop=True)
df = df.drop_duplicates(subset=['sampTime'], ignore_index=True)

# Preprocess, extract fixations and add them to the dataframe

In [39]:
sys.path.append('./FixationDetection')
from I2MC import runI2MC

# Path to data folders
# path_to_folders = 'C:/Users/artem/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_MullerLyer'
path_to_folders = 'D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_MullerLyer'

# get all folder names
folder_names = os.listdir(path_to_folders)

# read and process original datafile for each participant
for fn in folder_names:
    path = os.path.join(path_to_folders, fn, fn+'_record.csv')       
        
    df = pd.read_csv(path)        

# order frames and drop duplicate samples (with same sampleTime)
df = df[df.fName.notna()]
df = df[df.frameNr.notna()]
df = df.apply(pd.to_numeric, errors='ignore')

df = df.sort_values('frameNr')
df = df.reset_index(drop=True)
# df = df.drop_duplicates(subset=['user_pred_px_x', 'user_pred_px_y'], ignore_index=True)
df = df.drop_duplicates(subset=['sampTime'], ignore_index=True)

# get fixations for the original datafile for each participant
fixDF = runI2MC(path, plotData = False)

# add extracted fixations to the original data file (two new columns)
# for each timestamp where fixation was detected, FixXPos and FixYPos are added
idx = 0 # index of fixDF
FixXPos = np.zeros(df.shape[0])
FixYPos = np.zeros(df.shape[0])
FixStartEnd = np.empty(df.shape[0], dtype='U10')
FixStartEnd.fill('') # explicitly fill the array (good practice)

DistFromPrevFix = np.zeros(df.shape[0])
prev_fix_x = False # keep track of xy when fixation ends
prev_fix_y = False

PrevFixSampTime = np.zeros(df.shape[0])
prev_fix_sampTime = 0

# iterate thru the original dataframe, thru each sample
for index, row in df.iterrows():
    
    # make sure not to iterate out of range
    if idx < fixDF.shape[0]:
        
        # go to next fixation when fixation ends
        if row['sampTime'] > np.array(fixDF.FixEnd)[idx]:
                idx += 1
        
        # make sure not to iterate out of range
        if idx < fixDF.shape[0]:
            
            # when samples are within fixation, accumulate FixXPos and FixYPos
            if row['sampTime'] >= np.array(fixDF.FixStart)[idx] and row['sampTime'] <= np.array(fixDF.FixEnd)[idx]:

                FixXPos[index] = (np.array(fixDF.XPos)[idx])
                FixYPos[index] = (np.array(fixDF.YPos)[idx])
            
            # label samples on which fixation starts and ends
            if row['sampTime'] == np.array(fixDF.FixStart)[idx]:             
                FixStartEnd[index] = 'fix_start'
                
                if prev_fix_x != False:
                    DistFromPrevFix[index] = np.sqrt((np.array(fixDF.XPos)[idx] - prev_fix_x)**2 
                                            + (np.array(fixDF.YPos)[idx] - prev_fix_y)**2)
                    PrevFixSampTime[index] = prev_fix_sampTime
            
            elif row['sampTime'] == np.array(fixDF.FixEnd)[idx]:                
                FixStartEnd[index] = 'fix_end'             
                                    
                prev_fix_x = np.array(fixDF.XPos)[idx]
                prev_fix_y = np.array(fixDF.YPos)[idx]
                prev_fix_sampTime = np.array(row['sampTime'])
                
       

 
# add fixations to original dataframe
df['FixXPos'] = np.array(FixXPos)
df['FixYPos'] = np.array(FixYPos)
df['FixStartEnd'] = FixStartEnd
df['DistFromPrevFix'] = DistFromPrevFix
df['PrevFixSampTime'] = PrevFixSampTime


# Extract only samples when the target was presented
df = df[df.event=='target_on']






Importing and processing: "D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_MullerLyer\2023_06_27_08_39_36\2023_06_27_08_39_36_record.csv"
	Searching for valid interpolation windows
	Replace interpolation windows with Steffen interpolation
	2-Means clustering started for averaged signal
	Determining fixations based on clustering weight mean for averaged signal and separate eyes + 2*std


I2MC took 6.735012054443359s to finish!


C:\Users\Artem\Anaconda3\envs\default\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Artem\Anaconda3\envs\default\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


# Get and plot saccade latency distribution

In [40]:
# Get saccade latency

# get the first time sample when the target is presented
b = df.drop_duplicates(subset=['trialNr'],  keep='first', ignore_index=True)

# extract the columns needed
b = b[['trialNr', 'sampTime']]

# rename the columns so they would be added
b.columns = ['trialNr', 'targSampTime']

# merge the target time into the main df (one time per trial)
df = pd.merge(df, b, on="trialNr")

# get samples where fixation ended
a = df[df.FixStartEnd == 'fix_start']

# get samples with large enough preceeding saccade
a = a[a.DistFromPrevFix > 300]

# computs first fixation duration
c = a.PrevFixSampTime - a.targSampTime
a['FixDur'] = c
# plot first fixation durations (saccade latencies)
plt.figure()
plt.style.use('ggplot')
plt.hist(c)
plt.xlabel('Sacade latency (ms)')

# save figure
# plt.savefig('./Figures_MullerLyer/saccade_latency.jpg', dpi=1000, pad_inches=0)


Text(0.5, 0, 'Sacade latency (ms)')

# 1D Plot to compare raw samples to fixations

### 1. Plot separately for target left and target right
### 2. while looping through each trial
    a. rescale time variable to start from 0
    b. plot all raw x
    c. plot all fixationXPos
    d. zeros indicate saccade/no fixation

In [41]:

conditions = ['right', 'left']


for cond in conditions:
    
    plt.figure()
    plt.style.use('ggplot')
    plt.title(f'Target {cond}')
    
    a = df[df.target==cond]

    for name, group in a.groupby('trialNr'):
        t = np.array(group.sampTime)
        t = t-t[0]
        raw_h = plt.scatter(t, group.user_pred_px_x, c='orange', alpha=0.5, edgecolors='black')
        fix_h = plt.scatter(t, group.FixXPos, c='blue', alpha=0.5, edgecolors='black')

    plt.plot(t, group.targetX, c='green', lw=1)
    plt.plot(t, group.fixationStimX, c='red', lw=1, linestyle='dashed')
    
    plt.xlabel('Time (ms)')
    plt.ylabel('Horizontal eye position (pixels)')
    
    plt.legend((raw_h, fix_h), ('raw samples', 'fixations'), scatterpoints=1)
    
    # save figure
#     plt.savefig(f'./Figures_MullerLyer/1D_{cond}.jpg', dpi=1000, pad_inches=0)


# 2D plot of fixations and raw samples

## 1. plot all raw x,y
## 2. plot all fixations x,y 


In [46]:
conditions = ['right', 'left']


for cond in conditions:
    
    plt.figure()
    plt.style.use('ggplot')
    plt.title(f'Target {cond}')
        
    # select condition
    a = df[df.target==cond]
            
    raw_h = plt.scatter(a.user_pred_px_x, a.user_pred_px_y, c='orange', alpha=0.5, edgecolors='black')
    
     # remove no fixations/saccades (zeros)
    fix_h = plt.scatter(a.FixXPos[a.FixXPos>0], a.FixYPos[a.FixYPos>0], c='blue', alpha=0.5, edgecolors='black')
    
    # plot target and fixation cirle
    plt.scatter(a.fixationStimX, a.fixationStimY, c='red')
    plt.scatter(a.targetX, a.fixationStimY, c='green')
    
    # plot target and fixation vertical lines
    plt.plot(np.ones(df.resY.iloc[0].astype('int')) * a.fixationStimX.iloc[0], np.arange(df.resY.iloc[0]), c='red', lw=1, linestyle='dashed')
    plt.plot(np.ones(df.resY.iloc[0].astype('int')) * a.targetX.iloc[0], np.arange(df.resY.iloc[0]), c='green', lw=1, linestyle='dashed')
    
    plt.xlim((0, df.resX.iloc[0]))
    plt.ylim((df.resY.iloc[0]), 0)
    
    plt.xlabel('Horizontal eye position (pixels)')
    plt.ylabel('Vertical eye position (pixels)')
    
    plt.legend((raw_h, fix_h), ('raw samples', 'fixations'), scatterpoints=1)
    
    # save figure
#     plt.savefig(f'./Figures_MullerLyer/2D_{cond}.jpg', dpi=1000, pad_inches=0)


# Compare landing position between outward and inward arrows

In [43]:
def plot_fixations(df, cond='', agg='median', title=''):
    
    plt.style.use('ggplot')
#     plt.grid(False)
#     plt.set_axis_bgcolor('white')
    
    
    out = df[df.condition=='arrowHeadsOutward']
    inw = df[df.condition=='arrowHeadsInward']

    if agg=='median':
        
        outX = out.FixXPos.median()
        outY = out.FixYPos.median()

        inwX = inw.FixXPos.median()
        inwY = inw.FixYPos.median()
        
    elif agg=='mean':
        
        outX = out.FixXPos.mean()
        outY = out.FixYPos.mean()

        inwX = inw.FixXPos.mean()
        inwY = inw.FixYPos.mean()
        
        

    plt.figure()    
    plt.title(title);
    
    
    out_h = plt.scatter(out.FixXPos, out.FixYPos, c='blue', alpha=0.5, edgecolors='black')
    inw_h = plt.scatter(inw.FixXPos, inw.FixYPos, c='orange', alpha=0.5,edgecolors='black')
    
    plt.scatter(out.fixationStimX, out.fixationStimY, c='red')
    plt.scatter(out.targetX, out.fixationStimY, c='green')
    
    
    # plot target and fixation vertical lines
    plt.plot(np.ones(df.resY.iloc[0].astype('int')) * df.fixationStimX.iloc[0], np.arange(df.resY.iloc[0]), c='red', lw=1, linestyle='dashed')
    plt.plot(np.ones(df.resY.iloc[0].astype('int')) * df.targetX.iloc[0], np.arange(df.resY.iloc[0]), c='green', lw=1, linestyle='dashed')    

    plt.scatter(outX, outY, c='purple', edgecolors='black')
    plt.scatter(inwX, inwY, c='darkorange', edgecolors='black')
    
    # plot inward and outward vertical lines
    plt.plot(np.ones(df.resY.iloc[0].astype('int')) * outX, np.arange(df.resY.iloc[0]), c='purple', lw=1, linestyle='dashed')
    plt.plot(np.ones(df.resY.iloc[0].astype('int')) * inwX, np.arange(df.resY.iloc[0]), c='darkorange', lw=1, linestyle='dashed')    
    
    plt.xlim((0, df.resX.iloc[0]))
    plt.ylim((df.resY.iloc[0], 0))
    
    plt.xlabel('Horizontal eye position (pixels)')
    plt.ylabel('Vertical eye position (pixels)')
    
    plt.legend((out_h, inw_h), ('outward arrowheads', 'inward arrowheads'), scatterpoints=1)
    
    
    plt.text(150,700,f'Outward Arrow Xpos: {np.round(outX,1)}')
    plt.text(150,750,f'Inward Arrow Xpos: {np.round(inwX, 1)}')
    
    # save figure
#     plt.savefig(f'./Figures_MullerLyer/{cond}_OutInw.jpg', dpi=1000, pad_inches=0)


    return outX, outY, inwX, inwY



In [44]:
# Min distance to target in pixes
pix_in_cm = df.resX.iloc[0]/df.scrW_cm.iloc[0]
minDistTarg = pix_in_cm * 3

# get samples where fixation ended
a = df[df.FixStartEnd == 'fix_start']

# get samples with large enough preceeding saccade
a = a[a.DistFromPrevFix > 300]

# computs first fixation duration
c = a.PrevFixSampTime - a.targSampTime
a['FixDur'] = c

Left = a[a.target=='left']
out = Left[Left.condition=='arrowHeadsOutward']
inw = Left[Left.condition=='arrowHeadsInward']
print(f'Target Left Outward Arrow: {out.FixXPos.mean()}')
print(f'Target Left Inward Arrow: {inw.FixXPos.mean()}')

Right = a[a.target=='right']
out = Right[Right.condition=='arrowHeadsOutward']
inw = Right[Right.condition=='arrowHeadsInward']
print(f'Target Right Outward Arrow: {out.FixXPos.mean()}')
print(f'Target Right Inward Arrow: {inw.FixXPos.mean()}')


# Plot
summary_right = plot_fixations(Right, cond='right', agg='mean', title='Target Right')
summary_left = plot_fixations(Left, cond='left', agg='mean', title='Target Left')



Target Left Outward Arrow: 447.0562401432419
Target Left Inward Arrow: 439.40176595234857
Target Right Outward Arrow: 839.5617558275407
Target Right Inward Arrow: 851.2056021970429


### Plot heatmaps per each condition
Gaze positions for all subjects are combined

In [45]:
# Import fixations into output file




### To do
1. Summarize the incomplete datasets (failed calibration, other reasons)